# Data processing - Level 0

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib ipympl

In [ ]:
import logging
from helikite.constants import constants

logging.basicConfig(level=logging.INFO, format='%(message)s')

# define your input and output directories here
input_dir = constants.FLIGHT_DIRPATH
output_level0_dir = constants.OUTPUTS_FOLDER / "Processing" / "Level0"
output_level0_dir.mkdir(parents=True, exist_ok=True)

### Load flight data

Cleaner detects instruments automatically based on the output schema and the files in the input folder. However, it is possible to specify the list of instruments to be used explicitly.

Output schema contains the list of reference instrument candidates, i.e. for ORACLES campaign it is `[flight_computer_v2, smart_tether, pops]`.
The first available instrument from the list will be used as a reference instrument, unless `reference_instrument` is specified explicitly.

If the reference instrument is different from the flight computer and its time is shifted relative to the real time, you can specify `reference_instrument_shift`.

In [ ]:

from helikite.classes.output_schemas import OutputSchemas
from helikite import Cleaner

cleaner = Cleaner(
    output_schema=OutputSchemas.from_name(constants.OUTPUT_SCHEMA),
    input_folder=input_dir,
    flight_date=constants.FLIGHT_DATE,
    flight=constants.FLIGHT,
    # instruments = [...]
    # reference_instrument=pops,
    # reference_instrument_shift="-50s",
)

In [ ]:
cleaner.set_time_as_index()
cleaner.fill_missing_timestamps(cleaner.reference_instrument, freq="1s", fill_method="ffill")
cleaner.data_corrections()
cleaner.set_pressure_column()

### Cross correlation
Time synchronisation based on the pressure measured by the different instruments.

In [ ]:
cleaner.pressure_based_time_synchronization()
cleaner.plot_pressure()

**If needed, manually shift instrument by X seconds**

In [ ]:
#cleaner.smart_tether.df = cleaner.smart_tether.df.shift(freq="-2s")       # ST
#cleaner.pops.df = cleaner.pops.df.shift(freq="-50s")                      # POPS
#cleaner.msems_readings.df = cleaner.msems_readings.df.shift(freq="-7s")   # mSEMS readings
#cleaner.msems_inverted.df = cleaner.msems_inverted.df.shift(freq="-14s")  # mSEMS inverted
#cleaner.msems_scan.df = cleaner.msems_scan.df.shift(freq="-14s")          # mSEMS scan
#cleaner.mcda.df = cleaner.mcda.df.shift(freq="-1s")                       # mCDA

#cleaner.plot_pressure()

### Flight time definition
Chose start and end of the flight.
2 cases for ORACLES :
- Start and stop at sledge height
- Start and stop at balloon height (addition of 3.5 m later in the processing)

In [ ]:
cleaner.define_flight_times()

In [ ]:
cleaner.state()

### Dataframe creation
Removal of duplicates and merging instruments into one dataframe.

In [ ]:
cleaner.remove_duplicates()
cleaner.merge_instruments()

### Overview plot of time synchronisation

In [ ]:
from helikite.instruments import msems_scan

save_path = output_level0_dir / f"Level0_{constants.FLIGHT_BASENAME}_Flight_{constants.FLIGHT}_TimeSync.png"
cleaner.plot_time_sync(save_path, skip=[msems_scan])

## Level 0

In [ ]:
cleaner.shift_msems_columns_by_90s()

In [ ]:
cleaner.export_data(filepath=output_level0_dir / f"level0_{constants.FLIGHT_BASENAME}")